In [1]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode
from qdrant_client.http.models import Distance, VectorParams
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLSectionSplitter
from langchain.storage import InMemoryStore, LocalFileStore
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage._lc_store import create_kv_docstore
from astro_chart import full_chart_generation, get_astro_data


/home/mameuio/miniconda3/LLM/zhiweidoushu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Seach Chinese embedding model  
# import json
# for model in TextEmbedding.list_supported_models():
#     if "Chinese" in model["description"]:
#         print(json.dumps(model, indent=2))


# Embedding model 
model_handle = "jinaai/jina-embeddings-v2-base-zh"
embeddings = FastEmbedEmbeddings(model_name=model_handle)
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/BM25")

Fetching 18 files: 100%|██████████| 18/18 [00:01<00:00, 17.89it/s]


In [ ]:
#Create vector store and doc store
url = "http://localhost:6333"
collection_name = "zhiwei_DAG"

# #for 1st time
# docs = []  # put docs here
# client  = QdrantClient('http://localhost:6333')

# vectorstore = QdrantVectorStore.from_documents(
#       docs,
#       embeddings,
#       sparse_embedding=sparse_embeddings,
#       url=url,
#       prefer_grpc=True,
#       collection_name=collection_name,
#       retrieval_mode=RetrievalMode.HYBRID
#   )


client = QdrantClient(url=url, prefer_grpc=True)
vectorstore = QdrantVectorStore(
    embedding=embeddings,
    client=client,
    collection_name=collection_name,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID
)


fs = LocalFileStore("./store_location")
doc_store = create_kv_docstore(fs)

In [ ]:
#Chunking
# Read html file (actually not need but I just pratice)
with open('ziweidoushu book2.html','r',encoding='utf-8') as  f_in:
    text_html = f_in.read()

# Define the Splitters ---
# Parent chunk splitter
headers_to_split_on = [("h1", "Header 1")]
html_splitter = HTMLSectionSplitter(headers_to_split_on)
parent_docs  = html_splitter.split_text(text_html)
#parent_docs = parent_docs[0:10]

# Child chunk splitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size=300)
splits = child_splitter.split_documents(parent_docs )

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=doc_store,
    child_splitter=child_splitter,
    search_kwargs={"k": 5}
)

In [8]:
#Add Document
#retriever.add_documents(parent_docs)

In [9]:
query = "兄弟"
found_docs = retriever.invoke(query)

In [10]:
found_docs

[Document(metadata={'Header 1': '☆19、左辅星、右弼星(命宫)'}, page_content='☆19、左辅星、右弼星(命宫) \n 辅弼二星不是正曜，只是助星，基本性质为先天助力，平辈贵人，助力来自平辈，如同事、同学、朋友、兄弟、合作伙伴，亦可为晚辈。助力是天然的，不必刻意花力气去寻找。左辅较成熟稳重，偏向跑腿；右弼较温柔机谋，偏向顾问幕宾，带不爱约束和桃花色彩，左辅则无。左辅力量较右弼强，故命坐左辅会右弼比命坐右弼会左辅力强。 \n 左右二星无庙陷之分，落诸宫均能降福，四墓宫最佳。二星有多次之意，好坏均主发生多次。命宫有左或右，喜开车。 \n 左右亦宜双星同加会于三方四正宫才有力，只见单星则力弱或主代理工作。左或右单守命宫，命宫无正曜，无自主性，易受人左右，重感情，多主少年离家，或主离宗庶出、过房、随母改嫁、或入赘妻家，若对宫为同巨、阳梁、机阴、机巨、同阴的组合更是，若三方紫府昌曲魁钺合照，仍吉。六亲宫只加会单星，则易有双重的六亲，例如入父母宫主双重父母，入夫妻有二次婚姻，再见煞或加桃花星始验。 \n 左右均主乐观仁厚，即使落宫主星刚克或消极，亦能有所缓解而减轻。 \n 左右若与紫府同宫或拱照或相夹，是良臣逢明主，可达很高 \n 的成就。二星均不喜梁同机巨武等正星。逢天梁不羁，天同爱享受，天机善变，巨门是非，武曲决绝，与左右的本性不相洽，虽能减缓其弱点，但助力因而减弱，或增加感情困扰和波折。二星亦忌与杀破狼同宫，易有感情困扰，婚姻波折。读书限内遇左右，容易补考重考辍学等。逢化科，善思考，善策划，有声誉。 \n 辅弼分居于命迁宫者，终身福厚；分守命身宫亦吉，在命宫或三方照会稍次。分守命夫宫，无煞亦吉，有煞婚姻不顺。 \n 左右夹命为贵格，如安命丑宫，九月十一月生人；安命未官，三月五月生人，得左右夹命，不富则贵，不是这两月生人亦吉。亦宜主星庙旺无煞，不然不贵，仅主有人缘而已。紫破、天府、阴阳等组合，最喜左右夹，能增加地位和人生的稳定性。煞忌逢左右夹，亦可变不利为有利。若同时得龙池凤阁夹力量更强。 \n 左右同宫守命，披罗衣紫，辰戌宫安命，逢正月七月生人；丑宫安命，九月生人；巳宫安命，四月生人；卯亥宫安命，六月十二月生人，即是。不是这几个月生人亦吉。主得助力，事业顺遂。为人忠厚老诚，逢吉终生福厚寿长。 \n 命坐辰戌，得左右对照助力

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# Import relevant functionality
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3
)   



In [22]:
from langchain.prompts import PromptTemplate

system_prompt_text = """
#### 1. 角色
你是一个专业的紫微斗数**问题分析与查询生成引擎**。

#### 2. 核心任务
你的任务**不是**直接回答用户的提问。相反，你需要根据用户提供的**“原始问题”**和**“命盘信息”**，生成一个结构化的**“关键问题列表”**。这个列表将作为精确的查询指令 (Search Queries)，用于从一个基于《紫微斗数精成》第八、九章内容的 Vector Space 中提取信息。

#### 3. 输入格式
你将接收到两部分信息：
1.  **原始问题**: 用户提出的一个比较宽泛的问题 (例如：“我的财运怎么样？” 或 “我的感情顺利吗？”)。
2.  **命盘信息**: 一个包含十二宫位及其内部所有主星和重要辅星的结构化数据。


#### 4. 执行流程
1.  **解析核心宫位:**
    * 首先，分析用户的“原始问题”，识别其背后对应的核心**宫位**。
    * **对应关系示例**:
        * “我这个人怎么样？”、“我的性格”、“我的命运” -> **命宫**
        * “感情”、“婚姻”、“另一半” -> **夫妻宫**
        * “事业”、“工作”、“官运” -> **事业宫 (官禄宫)**
        * “财运”、“赚钱能力”、“财务状况” -> **财帛宫**
        * “出门在外”、“人际关系”、“机遇” -> **迁移宫**

2.  **提取关键星曜:**
    * **请生成只对核心宫位提出问题**
    * **空宫特别指令**: 在提取星曜前，首先检查核心宫位是否为“空宫”（即宫位内没有列出任何主星）。
    * **如果核心宫位是空宫，则必须将其对宫（根据下面的“对宫关系”查找）的主星视为本宫的核心分析星曜。**
    * 如果核心宫位非空宫，则提取该宫位内的所有**主星**和具有显著影响的**辅星**（特别是六吉星与六煞星）。
    **对宫关系**
    这是一个固定规则，用于查找任何宫位的对宫：
      * 命宫的对宫迁移宫
      * 兄弟宫的对宫交友宫
      * 夫妻宫的对宫官禄宫
      * 子女宫的对宫田宅宫
      * 财帛宫的对宫福德宫
      * 疾厄宫的对宫父母宫

3.  **生成关键问题列表:**
    * 基于提取出的宫位和星曜组合，生成一系列具体、明确、可用于检索的“关键问题”。
    * **生成模板**:
        * **单一星曜查询**: `[宫位名称] 中 [星曜A] 的解释。`
        * **空宫时星曜查询**: `  [星曜C([对宫的星曜])] 对 [本宫名称] 的影响。`



#### 5. 输出格式
你必须以一个清晰、编号的列表形式，输出所有生成的“关键问题”。
- 仅输出 JSON 字符串数组；不含任何解释、前后缀、注释或代码块标记。
- 项目去重、语义多样化、按相关性降序。

####
- 用户原始问题: {user_question}
- 用户命盘信息: {user_stars}

"""


ziwei_prompt_template_classic = PromptTemplate(
    input_variables=["user_question","user_stars"],
    template=system_prompt_text
)


chain = ziwei_prompt_template_classic | llm


In [23]:
question = "我什么时候能遇到正缘？"
stars = full_chart_generation("1994-8-25", 11, "男")

ai_message = chain.invoke({"user_question": question,"user_stars":  stars})

In [24]:
print(ai_message.content)

[
    "夫妻宫 中 天钺 的解释。",
    "天相 对 夫妻宫 的影响。",
    "夫妻宫 中 天姚 的解释。",
    "夫妻宫 中 天才 的解释。",
    "夫妻宫 中 天官 的解释。",
    "夫妻宫 中 天德 的解释。",
    "夫妻宫 中 寡宿 的解释。"
]


In [25]:
import json

def query_extraction(raw_output_string):
    start_index = raw_output_string.find('[')
    end_index = raw_output_string.rfind(']')

    # 如果找到了 '[' 和 ']'，就提取它们之间的内容
    if start_index != -1 and end_index != -1:
        clean_json_string = raw_output_string[start_index : end_index + 1]
        try:
            # 2. 使用 json.loads() 将清理后的字符串解析为 Python 列表
            queries_list = json.loads(clean_json_string)
            return queries_list
        except json.JSONDecodeError as e:
            print(f"解析失败: {e}")

    else:
        print("在字符串中没有找到有效的 JSON 数组 ('[...]')。")
        return []


In [26]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank
from langchain_openai import ChatOpenAI

In [ ]:
query = query_extraction(ai_message.content)

['夫妻宫 中 天钺 的解释。',
 '天相 对 夫妻宫 的影响。',
 '夫妻宫 中 天姚 的解释。',
 '夫妻宫 中 天才 的解释。',
 '夫妻宫 中 天官 的解释。',
 '夫妻宫 中 天德 的解释。',
 '夫妻宫 中 寡宿 的解释。']

In [37]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.document_compressors.rankllm_rerank import RankLLMRerank

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
test123 = compression_retriever.invoke(query[0])

In [43]:
query[0]

'夫妻宫 中 天钺 的解释。'

In [39]:
test123

[Document(metadata={'id': 2, 'relevance_score': np.float32(0.99872833), 'Header 1': '☆27、龙池、凤阁星'}, page_content='☆27、龙池、凤阁星 \n 龙池凤阁均为文明之宿，主才艺、声名、科甲，增加人的灵巧，仅主清贵而非富贵，有攀龙附凤的运气。凤阁在女，浪荡不定；龙池在男，实在威猛。主艺术、建筑、营造等。二星喜夹府相，主增加风姿文采。二星亦喜合不喜分。 \n 龙池凤阁二星同宫较有力，夹命或夹官禄宫更吉；二星分值则力量不大，难以发挥。喜会昌曲魁钺，可增强力量，能充分表现文艺方面的才华，不逢煞会有突破性的成就，利考试、竞争。会贪巨机梁（任一个都算），主口才佳，可为教师之职。 \n 二星守命身，主人清奇秀丽，雅重温良。男逢加诸吉，多主攀龙附凤，功名显达，财禄丰厚。女命逢之，貌美如花，秀外慧 \n 中，旺夫益子，志节高超。若正星庙旺，灵感特别好，对学习五术和修炼玄学很有利。 \n 龙池凤阁可乏指高级宾馆。凤阁也主房屋。 \n ☆龙池星——阳水。主科甲，文章，才华，艺术。动物代表蛇。 \n 专辅天府星，真龙藏身之所，代表男子建功立业之星。在物主水池、江河湖泊。 \n 个性豪爽，稳重，心胸宽广，喜豪华，聪明文雅，善良，贤德，忠贞不移，风流浪漫而不下流，有点傲气，讲够高级享受，尤其是饮食享受。 \n 人的立业、守业、安身之处即在龙池。会吉可接触高层人物，本身的声誉也会因此而提高，有声誉有作为，一生财务情况良好。 \n 入命，攀龙附凤，结交权贵，安份守业，善守业，有一定的经营市场，但进取力不足，须依赖他人之力来成就自己。男可安度晚年，有功名。亦主其人命中先天有来头。加煞冲会易有耳部毛病，若疾厄宫又不好者，晚年易有耳聋或重听；与七杀同度于命宫或疾厄宫，逢流煞冲起，亦是。与天府同宫，增加天府的吉力，有享受。 \n 入身，身体处于侍官之位，可与高人长官相伴，男子遇之自身乘龙气，女子遇之夫君显贵。 \n 女命遇之，长相清秀，聪明能干，忠贞不移，荣夫益子，长男可建功立业在龙池位。 \n ☆凤阁星——阳土。主科甲，文章，才华，艺术。 \n 专辅天相星，为凤的栖身之处，代表女子建功立业之星。代表飞鸟、楼阁、建筑物。 \n 为命宫主星时，面黄白色，方长脸，中矮身材，眉清目秀，风度优雅，高贵，有文士

In [40]:
def pretty_print_docs(docs):
    doc_text = f"\n{'-' * 100}\n".join(
        [f"Document {i + 1}:\n{d.page_content}" for i, d in enumerate(docs)]
    )
    print(doc_text)
    return doc_text

In [ ]:
answer_prompt_text = """
角色: 你是专业紫微斗数顾问，任务是基于检索到的文段整理关键信息，供后续回答使用。

目标: 在不直接回答用户问题的前提下，先归纳文段中与命盘,提问相关的要点。

输入:
- 用户命盘: {user_stars}
- 用户问题: {user_question}
- 检索文段: {retrieved_passages}

指令:
1. 阅读全部文段，挑出与用户问题及命盘相关的关键信息；忽略无关材料。
2. 将信息按主题分组，并以精炼语句概述
3. 指出这些要点与命盘要素（宫位、主星、三方四正、四化等）的关联，或说明缺少对应信息。
4. 如存在重要空缺或需要额外检索的线索，单独列出。
5. 不做结论或建议；不要编造命盘或文段外的信息。

输出格式:
关键信息:
- … 
- …
命盘关联:
- …
- …
信息缺口:
- …
"""


In [42]:
retrieved_passages = pretty_print_docs(test123)

Document 1:
☆27、龙池、凤阁星 
 龙池凤阁均为文明之宿，主才艺、声名、科甲，增加人的灵巧，仅主清贵而非富贵，有攀龙附凤的运气。凤阁在女，浪荡不定；龙池在男，实在威猛。主艺术、建筑、营造等。二星喜夹府相，主增加风姿文采。二星亦喜合不喜分。 
 龙池凤阁二星同宫较有力，夹命或夹官禄宫更吉；二星分值则力量不大，难以发挥。喜会昌曲魁钺，可增强力量，能充分表现文艺方面的才华，不逢煞会有突破性的成就，利考试、竞争。会贪巨机梁（任一个都算），主口才佳，可为教师之职。 
 二星守命身，主人清奇秀丽，雅重温良。男逢加诸吉，多主攀龙附凤，功名显达，财禄丰厚。女命逢之，貌美如花，秀外慧 
 中，旺夫益子，志节高超。若正星庙旺，灵感特别好，对学习五术和修炼玄学很有利。 
 龙池凤阁可乏指高级宾馆。凤阁也主房屋。 
 ☆龙池星——阳水。主科甲，文章，才华，艺术。动物代表蛇。 
 专辅天府星，真龙藏身之所，代表男子建功立业之星。在物主水池、江河湖泊。 
 个性豪爽，稳重，心胸宽广，喜豪华，聪明文雅，善良，贤德，忠贞不移，风流浪漫而不下流，有点傲气，讲够高级享受，尤其是饮食享受。 
 人的立业、守业、安身之处即在龙池。会吉可接触高层人物，本身的声誉也会因此而提高，有声誉有作为，一生财务情况良好。 
 入命，攀龙附凤，结交权贵，安份守业，善守业，有一定的经营市场，但进取力不足，须依赖他人之力来成就自己。男可安度晚年，有功名。亦主其人命中先天有来头。加煞冲会易有耳部毛病，若疾厄宫又不好者，晚年易有耳聋或重听；与七杀同度于命宫或疾厄宫，逢流煞冲起，亦是。与天府同宫，增加天府的吉力，有享受。 
 入身，身体处于侍官之位，可与高人长官相伴，男子遇之自身乘龙气，女子遇之夫君显贵。 
 女命遇之，长相清秀，聪明能干，忠贞不移，荣夫益子，长男可建功立业在龙池位。 
 ☆凤阁星——阳土。主科甲，文章，才华，艺术。 
 专辅天相星，为凤的栖身之处，代表女子建功立业之星。代表飞鸟、楼阁、建筑物。 
 为命宫主星时，面黄白色，方长脸，中矮身材，眉清目秀，风度优雅，高贵，有文士风度。无论男女喜修饰外表，讲够高级穿着享受，因而较懒散；个性浪漫，人缘广，清高，德高望重，动作敏捷，喜独往独来；聪明智慧，文笔好，具文字书画天才。 
 入命，攀龙附凤，结交权贵，对绘画具有天赋，再会甲级科名之星，其知名度颇高。对异性

In [44]:
summary_prompt  = PromptTemplate(
    input_variables=["user_question","user_stars","retrieved_passages"],
    template=answer_prompt_text
)
chain = summary_prompt  | llm


In [45]:
summaries = chain.invoke({"user_question": question,"user_stars":  stars,"retrieved_passages":retrieved_passages})
summary_text = summaries.content

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [48]:
print(summary_text)

关键信息:
- 夫妻宫的星曜组合可以揭示配偶的容貌、性格和才能，以及婚姻的好坏。
- 夫妻宫若有桃花星，可能会导致婚姻不美，且配偶易有外遇。
- 夫妻宫的吉星（如紫府昌曲等）坐守时，婚姻多和谐，能得助益；若有煞星则主婚姻不美。
- 夫妻宫与命宫的关系密切，命宫有不利星曜时，婚姻可能会受到影响。
- 夫妻宫的星曜组合与命宫的星曜组合相互影响，若夫妻宫强于命宫，配偶的能力和家世可能优于自己。

命盘关联:
- 用户命盘中夫妻宫为辛未，主星未见，辅星有天钺，可能影响配偶的性格和婚姻状况。
- 夫妻宫的星曜组合与命宫的星曜（癸酉）相互作用，可能影响婚姻的稳定性。
- 夫妻宫的吉凶情况与命宫的主星（廉贞、破军）及其状态（平、陷）相关。

信息缺口:
- 夫妻宫的具体星曜组合及其状态（如是否有煞星）未详细列出，需进一步分析。
- 需要了解用户的流年运势，以判断何时可能遇到正缘。
- 夫妻宫与命宫的三方四正情况未提供，可能影响婚姻的整体状况。


In [46]:
answer_prompt_text = """
角色: 你是专业紫微斗数顾问，需要结合用户命盘与摘要要点作答。

输入:
- 用户命盘: {user_stars}
- 用户问题: {user_question}
- 摘要要点: {summary_text}

指令:
1. 基于摘要要点,结合命盘结构说明关键影响,如果摘要的信息没办法回答请用你多年紫薇斗数的经验来回答问题
2. 明确回答用户问题，分条阐述理由。
3. 请尽量给出谨慎、可执行的建议
4. 全文使用中文，条理清晰。

输出格式:
最终结论: …
关键理由:
- …
- …
建议:
- …
"""

answer_prompt = PromptTemplate(
    input_variables=["user_question", "user_stars", "summary_text"],
    template=answer_prompt_text,
)
answer_chain = answer_prompt | llm

final_answer = answer_chain.invoke({
    "user_question": question,
    "user_stars": stars,
    "summary_text": summary_text,
})
print(final_answer.content)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


最终结论: 你在未来的几年内有机会遇到正缘，尤其是在流年运势较好的时候。

关键理由:
- **夫妻宫分析**: 你的夫妻宫为辛未，主星未见，辅星有天钺。天钺星在命盘中通常代表着贵人相助，可能会在你遇到正缘时提供支持和帮助。然而，夫妻宫的主星未见，可能意味着在感情方面会有一些波折，或者需要更多的努力去维持关系。
  
- **命宫与夫妻宫的关系**: 你的命宫为癸酉，主星廉贞和破军。廉贞星在命盘中通常代表着个性强烈、追求自由，而破军则可能带来一些不稳定因素。这种组合可能会影响你的婚姻稳定性，尤其是在感情的选择上可能会比较犹豫。

- **流年运势**: 根据你的命盘，未来的流年中，特别是在34~43虚岁（田宅宫大限）和44~53虚岁（官禄宫大限）期间，可能会有较好的机会遇到正缘。这些大限期间的运势较为旺盛，有助于你在感情方面的积极发展。

建议:
- **提升自我**: 在等待正缘的同时，建议你多关注自身的成长与提升，增强个人魅力和能力，这样在遇到合适的人时，能够更好地把握机会。
  
- **社交活动**: 积极参与社交活动，扩大交际圈，增加遇到正缘的机会。可以尝试参加一些兴趣小组或社交活动，增加与他人交流的机会。

- **关注流年**: 特别关注未来的流年运势，尤其是在34~43虚岁和44~53虚岁期间，适时调整自己的情感策略，抓住机会。

- **保持开放心态**: 在感情方面保持开放的心态，接受不同的人和事物，可能会有意想不到的收获。
